### Registration

**Learning outcomes:**
- How to apply registration to an image using ants.

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


In [3]:
raw_img_path = os.path.join(BASE_DIR,'Brain_Extraction', 'sub-01', 'anat', 'sub-01_T1w.nii')

#### Raw image

In [4]:
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

explore_3D_array(arr=raw_img_ants.numpy())

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Template image

In [5]:
template_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
template_img_ants = ants.image_read(template_img_path, reorient='IAL')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [6]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(template_img_ants)


		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 300, 208)
	 Spacing    : (0.8, 0.8, 0.8)
	 Origin     : (87.6611, -140.5019, -153.1909)
	 Direction  : [-0.0108 -0.0341 -0.9994  0.0195  0.9992 -0.0343  0.9998 -0.0199 -0.0101]

		TEMPLATE IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]



#### Registration

In [7]:
transformation = ants.registration(
    fixed=template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [00000236B932A690,00000236B932A860,1] -m mattes[00000236B932A690,00000236B932A860,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[00000236B932A690,00000236B932A860,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\Windows\AppData\Local\Temp\tmp46_dw6dp,00000236B932A5C0,00000236B932A590] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [8]:
print(transformation)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 300, 208)
	 Spacing    : (0.8, 0.8, 0.8)
	 Origin     : (87.6611, -140.5019, -153.1909)
	 Direction  : [-0.0108 -0.0341 -0.9994  0.0195  0.9992 -0.0343  0.9998 -0.0199 -0.0101]
, 'fwdtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp46_dw6dp1Warp.nii.gz', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp46_dw6dp0GenericAffine.mat'], 'invtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp46_dw6dp0GenericAffine.mat', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp46_dw6dp1InverseWarp.nii.gz']}


In [9]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array(arr=registered_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [10]:
out_folder =  os.path.join(BASE_DIR, 'Brain_Extraction', 'preprocessed')
out_folder = os.path.join(out_folder, 'Preprocessed_image') # create folder with name of the raw file
os.makedirs(out_folder, exist_ok=True) # create folder if not exists
os.makedirs(out_folder, exist_ok=True) # create folder if not exists

out_filename = add_suffix_to_filename('sub-01_T1w.nii', suffix='registration')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

\Brain_Extraction\sub-01\anat\sub-01_T1w.nii
\Brain_Extraction\preprocessed\Preprocessed_image\sub-01_T1w_registration.nii


In [11]:
registered_img_ants.to_file(out_path)